Images for the poster at the Nederlands Aardwetenschapplijk Congress, Utrecht, 14.3.19.

In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson, map_eq_earth
from GMST import GMST_timeseries, GMST_regression, atm_heat_content, GMST_GISTEMP
from paths import path_results, path_samoc, path_data
from plotting import shifted_color_map, discrete_cmap
from constants import abs_zero, cp_air
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import xr_AREA

In [ ]:
from statsmodels.tsa.arima_process import ArmaProcess
import mtspec
import matplotlib.ticker as mticker
from filters import lowpass
from regions import SST_index_bounds
import matplotlib.patches as mpatches

In [ ]:
hadcrut = xr.open_dataarray(f'{path_data}/HadCRUT/ihad4_krig_v2_0-360E_-90-90N_n_mean1_anom_30.nc', decode_times=False)
mpige = xr.open_dataset(f'{path_samoc}/GMST/GMST_MPI_GE.nc')
cmip5   = xr.open_dataarray(f'{path_data}/CMIP5/KNMI_CMIP5_GMST_yrly.nc', decode_times=False)  # first year 1861
mpige_1950_1980 = mpige.tsurf_mean[100:130].mean(dim='time')-273.15
cmip5_1950_1980 = cmip5[89:119].mean()
cmip5_1980_2010 = cmip5[119:149].mean()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8,5))
plt.tick_params(labelsize=20)
# plt.xlabel('time [year C.E.]', fontsize=20)
plt.ylabel('GMST anomaly [$^\circ$C]', fontsize=20)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
for i in range(100):
    L1, =plt.plot(np.arange(1850,2016), mpige.tsurf[i,:]-273.15-mpige_1950_1980,
             alpha=.2, lw=.5, c='grey', label='ensemble members')
# L2, = plt.plot(np.arange(1861,2019), cmip5[:158], lw=2, label='CMIP5 single member ensemble mean')

L3, = plt.plot(np.arange(1850,2016), mpige.tsurf_mean-273.15-mpige_1950_1980, lw=2, label='ensemble mean: forced signal')
L4, = plt.plot(np.arange(1850,2019), hadcrut, lw=2, label='observations: HadCRUT')
leg1 = plt.legend(handles=[L1, L3, L4], fontsize=20, frameon=False, loc=2)
plt.gca().add_artist(leg1)
plt.tight_layout()
plt.savefig(f'{path_results}/poster/GMST', dpi=600)

## Variance plots

### GMST

In [ ]:
import statsmodels.api as sm

In [ ]:
had_cmip5 = hadcrut[11:].values-cmip5[:158].values
# plt.plot(np.arange(1861,2019), had_cmip5, label=f'HadCRUT-CMIP5' )

def scaled_detrended_gmst(obs, mmem):
    assert len(obs)==len(mmem)

    X = mmem - np.mean(mmem)
    Y = obs - np.mean(obs)
#     plt.plot(X)
#     plt.plot(Y)
    model = sm.OLS(Y, X).fit()
    print(f'{len(obs)}, R^2: {model.rsquared:4.2e} params: {model.params}\n')
    a = model.params[0]
    return Y-a*X, a

had_cmip5, a_hc = scaled_detrended_gmst(hadcrut[11:].values, cmip5[:158].values  )

plt.plot(np.arange(1861,2019), had_cmip5, label=f'HadCRUT-{a_hc}*CMIP5' )
plt.legend()

In [ ]:
from OHC import replace_OHC_year
from filters import bandpass
from xr_regression import xr_lintrend, xr_quadtrend

In [ ]:
gmst_ctrl = xr.open_dataarray(f'{path_samoc}/GMST/GMST_yrly_ctrl.nc')
gmst_lpd  = xr.open_dataarray(f'{path_samoc}/GMST/GMST_yrly_lpd.nc' )

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_ctrl.nc'  , decode_times=False)
ctrl   = replace_OHC_year(ctrl  , 205)
lpd    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_lpd.nc'   , decode_times=False)

In [ ]:
def std_bandpass(tseries):
    if type(tseries)==xr.core.dataarray.DataArray:
        tseries = tseries-xr_quadtrend(tseries)
        tseries = tseries.values
    a = np.std(bandpass(tseries, np.array([20,5])))
    b = np.std(bandpass(tseries, np.array([100,20])))
    return a, b



f, ax = plt.subplots(2,2,figsize=(14,7),sharex='col', sharey=True)
for i in range(2):
    for j in range(2):
        ax[i,j].tick_params(labelsize=20)

        ax[i,j].spines['right'].set_visible(False)
        ax[i,j].spines['top'].set_visible(False)
    
    ax[i,0].set_yticks([2.75, 2, 1])
    ax[i,0].set_yticklabels(['obs', 'high\nres.', 'low\nres.'], fontsize=20)

ax[1,0].set_xlabel('GMST variance [K]', fontsize=30)

had1, had2 = std_bandpass(had_cmip5)
ctrl1, ctrl2 = std_bandpass(gmst_ctrl[-149:])
ctrl3, ctrl4 = std_bandpass(gmst_ctrl[:149])
lpd1, lpd2 = std_bandpass(gmst_lpd[-149:])
lpd3, lpd4 = std_bandpass(gmst_lpd[-2*149:-149])

ax[0,0].set_ylabel('decadal', fontsize=30)
ax[0,0].barh(2.75, had1 , 0.42, color='C1')
ax[0,0].barh(2.25, ctrl1, 0.42, color='C0')
ax[0,0].barh(1.75, ctrl3, 0.42, color='C0')
ax[0,0].barh(1.25, lpd1 , 0.42, color='C2')
ax[0,0].barh(0.75, lpd3 , 0.42, color='C2')


ax[1,0].barh(2.75, had2 , 0.42, color='C1')
ax[1,0].barh(2.25, ctrl2, 0.42, color='C0')
ax[1,0].barh(1.75, ctrl4, 0.42, color='C0')
ax[1,0].barh(1.25, lpd2,  0.42, color='C2')
ax[1,0].barh(0.75, lpd4,  0.42, color='C2')
ax[1,0].set_ylabel('multidecadal', fontsize=30)

ax[1,1].set_xlabel('global OHC variance [ZJ]', fontsize=30)

ctrl1, ctrl2 = std_bandpass(ctrl.OHC_global[-149:])
ctrl3, ctrl4 = std_bandpass(ctrl.OHC_global[:149])
lpd1, lpd2 = std_bandpass(lpd.OHC_global[-149:])
lpd3, lpd4 = std_bandpass(lpd.OHC_global[-2*149:-149])

ax[0,1].barh(2.25, ctrl1/1e21, 0.42, color='C0')
ax[0,1].barh(1.75, ctrl3/1e21, 0.42, color='C0')
ax[0,1].barh(1.25, lpd1 /1e21, 0.42, color='C2')
ax[0,1].barh(0.75, lpd3 /1e21, 0.42, color='C2')

ax[1,1].barh(2.25, ctrl2/1e21, 0.42, color='C0')
ax[1,1].barh(1.75, ctrl4/1e21, 0.42, color='C0')
ax[1,1].barh(1.25, lpd2 /1e21, 0.42, color='C2')
ax[1,1].barh(0.75, lpd4 /1e21, 0.42, color='C2')

print('test')
plt.tight_layout()
plt.savefig(f'{path_results}/poster/std_GMST_OHC', dpi=600)

### modes

In [ ]:
AMO_ctrl1 = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_151_299_ctrl.nc')
AMO_ctrl2 = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_100_248_ctrl.nc')
AMO_lpd1  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_268_416_lpd.nc' )
AMO_lpd2  = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_417_565_lpd.nc' )
AMO_had   = xr.open_dataarray(f'{path_samoc}/SST/AMO_GMST_dt_raw_had.nc' )

SOM_ctrl1 = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_151_299_ctrl.nc')
SOM_ctrl2 = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_100_248_ctrl.nc')
SOM_lpd1  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_268_416_lpd.nc' )
SOM_lpd2  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_417_565_lpd.nc' )
SOM_had   = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_had.nc' )

TPI_ctrl1 = xr.open_dataarray(f'{path_samoc}/SST/TPI_151_299_ctrl.nc')
TPI_ctrl2 = xr.open_dataarray(f'{path_samoc}/SST/TPI_100_248_ctrl.nc')
TPI_lpd1  = xr.open_dataarray(f'{path_samoc}/SST/TPI_268_416_lpd.nc' )
TPI_lpd2  = xr.open_dataarray(f'{path_samoc}/SST/TPI_417_565_lpd.nc' )
TPI_had   = xr.open_dataarray(f'{path_samoc}/SST/TPI_had.nc' )

In [ ]:
plt.figure(figsize=(7,7))

plt.bar(.7  , AMO_had  .std(), 0.12, color='C1')
plt.bar(.85 , AMO_ctrl1.std(), 0.12, color='C0')
plt.bar(1   , AMO_ctrl2.std(), 0.12, color='C0')
plt.bar(1.15, AMO_lpd1 .std(), 0.12, color='C2')
plt.bar(1.3 , AMO_lpd2 .std(), 0.12, color='C2')

plt.bar(1.7 , SOM_had  .std(), 0.12, color='C1')
plt.bar(1.85, SOM_ctrl1.std(), 0.12, color='C0')
plt.bar(2   , SOM_ctrl2.std(), 0.12, color='C0')
plt.bar(2.15, SOM_lpd1 .std(), 0.12, color='C2')
plt.bar(2.3 , SOM_lpd2 .std(), 0.12, color='C2')

plt.bar(2.7 , TPI_had  .std(), 0.12, color='C1')
plt.bar(2.85, TPI_ctrl1.std(), 0.12, color='C0')
plt.bar(3   , TPI_ctrl2.std(), 0.12, color='C0')
plt.bar(3.15, TPI_lpd1 .std(), 0.12, color='C2')
plt.bar(3.3 , TPI_lpd2 .std(), 0.12, color='C2')

plt.tight_layout()

In [ ]:
AMO_had.plot()
bandpass(AMO_had  , np.array([70,20])).plot()
AMO_ctrl1.plot()
bandpass(AMO_ctrl1, np.array([70,20])).plot()
AMO_lpd1.plot()
bandpass(AMO_lpd1, np.array([70,20])).plot()

In [ ]:
plt.figure(figsize=(7,7))

plt.bar(.7  , bandpass(AMO_had  , np.array([70,20])).std(), 0.12, color='C1')
plt.bar(.85 , bandpass(AMO_ctrl1, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(1   , bandpass(AMO_ctrl2, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(1.15, bandpass(AMO_lpd1 , np.array([70,20])).std(), 0.12, color='C2')
plt.bar(1.3 , bandpass(AMO_lpd2 , np.array([70,20])).std(), 0.12, color='C2')
plt.bar(1.7 , bandpass(SOM_had  , np.array([70,20])).std(), 0.12, color='C1')
plt.bar(1.85, bandpass(SOM_ctrl1, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(2   , bandpass(SOM_ctrl2, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(2.15, bandpass(SOM_lpd1 , np.array([70,20])).std(), 0.12, color='C2')
plt.bar(2.3 , bandpass(SOM_lpd2 , np.array([70,20])).std(), 0.12, color='C2')
plt.bar(2.7 , bandpass(TPI_had  , np.array([70,20])).std(), 0.12, color='C1')
plt.bar(2.85, bandpass(TPI_ctrl1, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(3   , bandpass(TPI_ctrl2, np.array([70,20])).std(), 0.12, color='C0')
plt.bar(3.15, bandpass(TPI_lpd1 , np.array([70,20])).std(), 0.12, color='C2')
plt.bar(3.3 , bandpass(TPI_lpd2 , np.array([70,20])).std(), 0.12, color='C2')

plt.tight_layout()

## Spectra

In [ ]:
AMO_ctrl = xr.open_dataarray(f'{path_samoc}/SST/AMO_raw_ctrl.nc')

TPI1_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI1_GMST_dt_raw_ctrl.nc')
TPI1_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI1_dt_raw_lpd.nc' )
TPI1_had  = xr.open_dataarray(f'{path_samoc}/SST/TPI1_dt_raw_had.nc' )

TPI2_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI2_GMST_dt_raw_ctrl.nc')
TPI2_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI2_dt_raw_lpd.nc' )
TPI2_had  = xr.open_dataarray(f'{path_samoc}/SST/TPI2_dt_raw_had.nc' )

TPI3_ctrl = xr.open_dataarray(f'{path_samoc}/SST/TPI3_GMST_dt_raw_ctrl.nc')
TPI3_lpd  = xr.open_dataarray(f'{path_samoc}/SST/TPI3_dt_raw_lpd.nc' )
TPI3_had  = xr.open_dataarray(f'{path_samoc}/SST/TPI3_dt_raw_had.nc' )

In [ ]:
AMO_ctrl.plot()
# AMO_lpd .plot()
# AMO_had .plot()

In [ ]:
ds = xr.open_dataarray(f'{path_samoc}/SST/TPI3_GMST_dt_raw_ctrl.nc')

In [ ]:
ds.plot()

In [ ]:
TPI_ctrl.plot()

In [ ]:
TPI3_ctrl[:149].plot()

In [ ]:
(TPI2_ctrl-(TPI1_ctrl+TPI3_ctrl)/2).plot()
TPI_ctrl.plot()

In [ ]:
SOM_ctrl.plot()
xr_lintrend(SOM_ctrl).plot()
(SOM_ctrl-xr_lintrend(SOM_ctrl)).plot()
# SOM_lpd .plot()
# SOM_had .plot()

In [ ]:
def autocorrelation(ts, n=1):
    """ calculates the first n lag autocorrelation coefficient """
    n += 1  # zeroth element is lag-0 autocorrelation = 1
    acs = np.ones((n))
    for i in np.arange(1,n):
        acs[i] = np.corrcoef(ts[:-i]-ts[:-i].mean(), ts[i:]-ts[i:].mean())[0,1]
    return acs

In [ ]:
def spectrum(data, filter_type=None, filter_cutoff=None):
    """ multitaper spectrum """
#     assert type(data)==np.ndarray
    if filter_type is not None:
        assert filter_type in ['lowpass', 'chebychev']
        assert type(filter_cutoff)==int
        assert filter_cutoff>1
        if filter_type=='lowpass':
            data = lowpass(data, filter_cutoff)
        elif filter_type=='chebychev':
            data = chebychev(data, filter_cutoff)

#     fit = np.polyfit(np.arange(len(data)), data, 1)
#     for t in range(149):
#         data[t] -= fit[0]*t + fit[1]
            
    spec, freq, jackknife, _, _ = mtspec.mtspec(
            data=data, delta=1., time_bandwidth=4,
            number_of_tapers=5, statistics=True)
    return (spec, freq, jackknife)

In [ ]:
def mc_ar1(ts, n=1000):
    """ Monte-Carlo AR(1) processes """
    phi = autocorrelation(ts)[1]
    AR_object = ArmaProcess(np.array([1, -phi]), np.array([1]))
    mc = np.zeros((n, len(ts)))
    for i in range(n):
        mc[i,:] = AR_object.generate_sample(nsample=len(ts))
        mc[i,:] *= np.std(ts)/np.std(mc[i,:])
    return mc

In [ ]:
def mc_ar1_spectrum(ts, n=1000, filter_type=None, filter_cutoff=None):
    """ calculates the MC avg spectrum and the 95% confidence interval """
    mc = mc_ar1(ts)
    if filter_type is not None:
        assert filter_type in ['lowpass', 'chebychev']
        assert type(filter_cutoff)==int
        assert filter_cutoff>1
        if filter_type=='lowpass':
            mc = lowpass(mc.T, filter_cutoff).T
        elif filter_type=='chebychev':
            mc = chebychev(mc.T, filter_cutoff).T

    mc_spectra = np.zeros((n, int(len(ts)/2)+1))
    for i in range(n):
        (mc_spectra[i,:], freq, jk) = spectrum(data=mc[i,:])
    mc_spectrum = np.zeros((4, int(len(ts)/2)+1))
    mc_spectrum[0,:] = np.median(mc_spectra, axis=0)
    mc_spectrum[1,:] = freq
    mc_spectrum[2,:] = np.percentile(mc_spectra,  5, axis=0)
    mc_spectrum[3,:] = np.percentile(mc_spectra, 95, axis=0)
    return mc_spectrum

In [ ]:
from xr_regression import xr_lintrend

In [ ]:
def plot_spectrum_ar1(ts, index, text, text2, obs=False):
    """ plots spectrum of single time series + AR(1) spectrum
    AR(1) spectrum includes uncertainties from MC simulation
    for the filtered SST indices, the AR(1) process is first fitted to the annual data
    """
    ts = ts[-149:]
    ts -= xr_lintrend(ts)
    ts = ts.values
 
    ft, fc = 'lowpass', 13
    ts_spectrum = spectrum(ts, filter_type=ft, filter_cutoff=fc)
    mc_spectrum = mc_ar1_spectrum(ts, filter_type=ft, filter_cutoff=fc)

    if obs==True:    fig, ax = plt.subplots(1, 1, figsize=(8,4))
    if obs==False:   fig, ax = plt.subplots(1, 1, figsize=(6.5,4))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
#     ax.get_xaxis().get_major_formatter().labelOnlyBase = False
    ax.tick_params(labelsize=20)
    ax.set_yscale('log')     
    ax.set_xscale('log')     
    L2 = ax.fill_between(1/mc_spectrum[1,:], mc_spectrum[2,:],  mc_spectrum[3,:],
                    color='C1', alpha=.3, label='5-95% C.I.')
    L1, = ax.plot(1/mc_spectrum[1,:], mc_spectrum[0,:], c='C1', label=f'red noise')     
    L4 = ax.fill_between(1/ts_spectrum[1], ts_spectrum[2][:, 0], ts_spectrum[2][:, 1],
                   color='C0', alpha=0.3, label=f'jackknife estimator')
    L3, = ax.plot(1/ts_spectrum[1], ts_spectrum[0], c='C0', label=f'{index} spectrum')
    ax.legend(handles=[L3, L4, L1, L2], fontsize=20, frameon=False, loc=4)
    ymax = 2
    ax.set_ylim((1E-6, ymax))
    ax.set_xlim((6, 150))
    ax.set_xticks([6, 8,10, 20, 40, 60, 80, 100, 150])
    ax.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())
    ax.text(7, .2, f'{index}: {text}', fontsize=30)
    ax.set_xlabel(r'Period [years]', fontsize=20)
    if obs==True: ax.set_ylabel(f'Spectral Power', fontsize=30)
    if obs==False: ax.set_yticks([])
    plt.tight_layout()
    plt.savefig(f'{path_results}/poster/{index}_spectrum_{text2}', dpi=800)

In [ ]:
texts = ['obs.', 'high res.', 'high res.', 'low res.', 'low res.']
texts2 = ['had', '151_299_ctrl', '100_248_ctrl', '268_416_lpd', '417_565_lpd']
obs = [True, False, False, False, False]

In [ ]:
for i, ts in enumerate([AMO_had, AMO_ctrl1, AMO_ctrl2, AMO_lpd1, AMO_lpd2]):
    plot_spectrum_ar1(ts, 'AMO', texts[i], texts2[i], obs[i])

In [ ]:
for i, ts in enumerate([SOM_had, SOM_ctrl1, SOM_ctrl2, SOM_lpd1, SOM_lpd2]):
    plot_spectrum_ar1(ts, 'SOM', texts[i], texts2[i], obs[i])

In [ ]:
for i, ts in enumerate([TPI_had, TPI_ctrl1, TPI_ctrl2, TPI_lpd1, TPI_lpd2]):
    plot_spectrum_ar1(ts, 'TPI', texts[i], texts2[i], obs[i])

In [ ]:
def plot_spectrum_ar1_TPI(tpi1, tpi2, tpi3, index):
    """ plots spectrum of single time series + AR(1) spectrum
    AR(1) spectrum includes uncertainties from MC simulation
    for the filtered SST indices, the AR(1) process is first fitted to the annual data
    """
    n=1000
    tpi = tpi2[-149:] - (tpi1[-149:]+tpi3[-149:])/2
    tpi -= xr_lintrend(tpi)

    mc = np.zeros((3, 1000, 149))
    for i, ts in enumerate([tpi1, tpi2, tpi3]):
        mc[i,:,:] = mc_ar1(ts.values[-149:])
    
    mc_tpi = mc[1,:,:] - (mc[0,:,:]+mc[2,:,:])/2
    for i in range(n):
        fit = np.polyfit(np.arange(149), mc_tpi[i,:], 1)
        for t in range(149):
            mc_tpi[i,t] -= fit[0]*t + fit[1]
        mc_tpi[i,:] *= np.std(tpi.values)/np.std(mc_tpi[i,:])
    mc = lowpass(mc_tpi.T, 13).T
    mc_spectra = np.zeros((n, int(149/2)+1))
    for i in range(n):
        (mc_spectra[i,:], freq, jk) = spectrum(data=mc[i,:])
    mc_spectrum = np.zeros((4, int(149/2)+1))
    mc_spectrum[0,:] = np.median(mc_spectra, axis=0)
    mc_spectrum[1,:] = freq
    mc_spectrum[2,:] = np.percentile(mc_spectra,  5, axis=0)
    mc_spectrum[3,:] = np.percentile(mc_spectra, 95, axis=0)
    
    ts_spectrum = spectrum(tpi.values, filter_type='lowpass', filter_cutoff=13)

    fig, ax = plt.subplots(1, 1, figsize=(8,5))
    ax.tick_params(labelsize=14)
    ax.set_yscale('log')     
    ax.set_xscale('log')     
    L2 = ax.fill_between(1/mc_spectrum[1,:], mc_spectrum[2,:],  mc_spectrum[3,:],
                    color='C1', alpha=.3, label='5-95% C.I.')
    L1, = ax.plot(1/mc_spectrum[1,:], mc_spectrum[0,:], c='C1', label=f'MC AR(1)')     
    L4 = ax.fill_between(1/ts_spectrum[1], ts_spectrum[2][:, 0], ts_spectrum[2][:, 1],
                   color='C0', alpha=0.3, label=f'jackknife estimator')
    L3, = ax.plot(1/ts_spectrum[1], ts_spectrum[0], c='C0', label=f'{index} spectrum')
    leg1 = plt.legend(handles=[L1, L2], fontsize=14, frameon=False, loc=3)
    ax.legend(handles=[L3,L4], fontsize=14, frameon=False, loc=1)
    ax.add_artist(leg1)
    ymax = 1e1
    if any([mc_spectrum[3,:].max()>ymax, ts_spectrum[2][:, 1].max()>ymax]):
        ymax = max([mc_spectrum[3,:].max(), ts_spectrum[2][:, 1].max()])
    ax.set_ylim((1E-6, ymax))
#     ax.set_xlim((0, 0.1))
    ax.set_xlim((6, 200))
#     ax.set_xticks([8,10, 20, 40, 60, 80, 100, 200], labels=[8,10, 20, 40, 60, 80, 100, 200])
#     ax.set_xlabel(r'Frequency [yr$^{-1}$]', fontsize=14)
    ax.set_xlabel(r'Period [yr]', fontsize=14)
    ax.set_ylabel(f'{index} Power Spectral Density', fontsize=14)
    plt.tight_layout()
#     plt.savefig(f'{path_results}/SST/{self.index}_AR1_spectrum_{run}')

In [ ]:
plot_spectrum_ar1_TPI(TPI1_had , TPI2_had , TPI3_had , 'TPI')
plot_spectrum_ar1_TPI(TPI1_ctrl, TPI2_ctrl, TPI3_ctrl, 'TPI')
plot_spectrum_ar1_TPI(TPI1_lpd , TPI2_lpd , TPI3_lpd , 'TPI')

## Maps

In [ ]:
from maps import rect_polygon

In [ ]:
def make_map(xa, domain, proj, cmap, minv, maxv, label, filename=None, text1=None, text2=None, rects=None, sig=None, clon=0):
    """ global map (Robinson or Equal Earth projection) of xa 
    optional: significance shading, polygons, text, central longitude, file output 
    """
    assert type(xa)==xr.core.dataarray.DataArray
    assert domain in ['atm', 'ocn_T', 'ocn_U', 'ocn_rect', 'ocn_low', 'ocn_had']
    assert proj in ['ee', 'rob']
    
    fig = plt.figure(figsize=(8,5))
    if proj=='ee':
        ax  = fig.add_subplot(1, 1, 1, projection=ccrs.EqualEarth(central_longitude=clon))
    elif proj=='rob':
        ax  = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson(central_longitude=clon))
    ax.set_position([.02,.05,.96,.93])
    cax, kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.03,shrink=0.8)
    
    print(domain)
    if domain in ['atm']:#, 'ocn_low']:
        lats = xa.lat
        lons = xa.lon
    elif domain in ['ocn_T', 'ocn_low']:
        lats = xa.TLAT
        lons = xa.TLONG
    elif domain=='ocn_U':
        lats = xa.ULAT
        lons = xa.ULONG
    elif domain=='ocn_rect':
        lats = xa.t_lat
        lons = xa.t_lon
    elif domain=='ocn_had':
        lats = xa.latitude
        lons = xa.longitude
        lons, lats = np.meshgrid(lons, lats)
    
    im = ax.pcolormesh(lons, lats, xa.values,
                       cmap=cmap, vmin=minv, vmax=maxv,
                       transform=ccrs.PlateCarree(),
                      )

    # significance outline
    if type(sig)==xr.core.dataarray.DataArray:
        if domain=='ocn_had':  flons, flats = lons.flatten(), lats.flatten()
        else:  flons, flats = lons.values.flatten(), lats.values.flatten()
#         flons, flats = lons.values.flatten(), lats.values.flatten()
        ax.tricontour(flons, flats, sig.values.flatten(), levels=[.5],
                      linestyles='dashed', linewidths=1.5, #cmap='gray',
                      transform=ccrs.PlateCarree(),
                     )
    
    # coastlines/land
    if domain=='atm':
        ax.coastlines()
    elif domain in ['ocn_T', 'ocn_U', 'ocn_had', 'ocn_low']:
        ax.add_feature(cartopy.feature.LAND,
                       zorder=2, edgecolor='black', facecolor='w')
        
    # text
    if text1!=None:
        ax.text(0, 1, text1, ha='left' , va='top', transform=ax.transAxes, fontsize=16, color='k')
    if text2!=None:
        ax.text(1, 1, text2, ha='right', va='top', transform=ax.transAxes, fontsize=16, color='k')
    
    # SST index polygons
    if type(rects)==np.ndarray:
        rects = [rects]
    elif type(rects)==list:
        for rect in rects:
            assert type(rect)==np.ndarray
            ax.add_patch(mpatches.Polygon(xy=rect,
                                          facecolor='none', edgecolor='k',
                                          linewidth=2, zorder=2,
                                          transform=ccrs.PlateCarree(),
                                         ),
                        )
            
    # grid
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = mticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = mticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    
    # colorbar
    cbar = fig.colorbar(im, cax=cax, extend='both', **kw)
    cbar.ax.tick_params(labelsize=14, color='k')
    label = cbar.set_label(label, size=16, color='k')
    
    # output
    if filename!=None: plt.savefig(filename, dpi=800, transparent=True)
    return fig, ax


In [ ]:
from regions import boolean_mask, SST_index_bounds
import cmocean

In [ ]:
def regr_map(ds, index, run, fn=None):
    """ map of regression slope with 95% significance countours and SST index polygons """
    if run in ['ctrl', 'rcp']:   domain = 'ocn'
    elif run in ['lpd', 'lpi']:  domain = 'ocn_low'
    elif run=='had':             domain = 'ocn_had'
    MASK = boolean_mask(domain=domain, mask_nr=0)
    
    xa = ds.slope.where(MASK)
    if domain in ['ocn', 'ocn_low']:
        xa = xa.assign_coords(TLONG=ds.TLONG)
    
    if index in ['AMO', 'SOM']:
        rects = [rect_polygon(SST_index_bounds(index))]
        clon = 300
        nv = .4
        print(rects)
    elif index in ['PDO', 'IPO']:
        rects = [rect_polygon(SST_index_bounds(index))]
        clon = 200
        print(rects)
        nv = .4
    elif index=='TPI':
        rects = [rect_polygon(SST_index_bounds('TPI1')),
                 rect_polygon(SST_index_bounds('TPI2')),
                 rect_polygon(SST_index_bounds('TPI3')),
                ]
        clon = 200
        nv = .4
    
    print(run, np.max(ds.slope.values))
    
    # choose two-tailed 95% significance level
    # as boolean map
    sig = ds.pval#.where(MASK)
#     tail1 = np.where(sig<0.025, 1, 0)
    tail1 = np.where(sig<0.005, 1, 0)
#     tail2 = np.where(sig>0.975, 1, 0)
    tail2 = np.where(sig>99.5, 1, 0)
    sig.values = tail1 + tail2
#     if run in ['ctrl', 'rcp', 'had']:   sig = sig.where(MASK)
    
    proj = 'rob'
    cm = discrete_cmap(16, cmocean.cm.balance)    
    label ='regression slope [K/std]'
    text2 = f'SST({index})\nregr.'
    if run=='had':  text1 = f'obs.'
    if run=='ctrl': text1 = f'high res.'
    if run=='lpd':  text1 = f'low res.'
    if fn==None:  filename = f'{path_results}/poster/{index}_regr_map_{run}.png'
    else:         filename = f'{path_results}/poster/{index}_regr_map_{run}_{fn}.png'
    if run in ['ctrl', 'rcp']:   domain = 'ocn_T'
        
    f, ax = make_map(xa=xa, domain=domain, proj=proj, cmap=cm, minv=-nv, maxv=nv,
                     label=label, filename=filename, text1=text1, text2=text2,
                     rects=rects, sig=sig, clon=clon)

In [ ]:
SST_index_bounds('AMO')

In [ ]:
SST_index_bounds('SOM')

In [ ]:
SST_index_bounds('TPI3')

In [ ]:
tslices = ['', '_151_299', '_268_416']
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    for index in ['AMO', 'SOM', 'TPI']:
        fn = f'{path_samoc}/SST/{index}_regr{tslices[i]}_{run}.nc'
        ds = xr.open_dataset(fn)
        regr_map(ds=ds, index=index, run=run, fn=tslices[i])

### variability map

In [ ]:
rects = [rect_polygon(SST_index_bounds('SOM')), rect_polygon(SST_index_bounds('AMO'))]

In [ ]:
fn = f'{path_samoc}/SST/SST_std_had.nc'
# fa = FieldAnalysis(SST_GMST_dt_yrly_had[-100:])
#     xa = fa.make_standard_deviation_map(fn=fn)
xa = xr.open_dataarray(fn)

fn = f'{path_results}/NAC_Poster/SST_std_map_obs'
label = 'standard deviation of detrended SST [K]'
cmap = 'viridis'
txt1 = f'HadISST'
txt2 = '1919-2018'
make_map(xa=xa, domain='ocn_had', proj='rob', cmap=cmap, minv=0, maxv=.7, rects=rects,
         clon=300, label=label, filename=fn, text1=txt1, text2=txt2)

In [ ]:
def std_bandpass(tseries):
    if type(tseries)==xr.core.dataarray.DataArray:
        tseries = tseries-xr_quadtrend(tseries)
        tseries = tseries.values
    a = np.std(bandpass(tseries, np.array([20,5])))
    b = np.std(bandpass(tseries, np.array([100,20])))
    return a, b



f, ax = plt.subplots(2,2,figsize=(14,7),sharex='col', sharey=True)
for i in range(2):
    for j in range(2):
        ax[i,j].tick_params(labelsize=20)

        ax[i,j].spines['right'].set_visible(False)
        ax[i,j].spines['top'].set_visible(False)
    
    ax[i,0].set_yticks([2.75, 2, 1])
    ax[i,0].set_yticklabels(['obs', 'high\nres.', 'low\nres.'], fontsize=20)

ax[1,0].set_xlabel('GMST variance [K]', fontsize=30)

had1, had2 = std_bandpass(had_cmip5)
ctrl1, ctrl2 = std_bandpass(gmst_ctrl[-149:])
ctrl3, ctrl4 = std_bandpass(gmst_ctrl[:149])
lpd1, lpd2 = std_bandpass(gmst_lpd[-149:])
lpd3, lpd4 = std_bandpass(gmst_lpd[-2*149:-149])

ax[0,0].set_ylabel('decadal', fontsize=30)
ax[0,0].barh(2.75, had1 , 0.42, color='C1')
ax[0,0].barh(2.25, ctrl1, 0.42, color='C0')
ax[0,0].barh(1.75, ctrl3, 0.42, color='C0')
ax[0,0].barh(1.25, lpd1 , 0.42, color='C2')
ax[0,0].barh(0.75, lpd3 , 0.42, color='C2')


ax[1,0].barh(2.75, had2 , 0.42, color='C1')
ax[1,0].barh(2.25, ctrl2, 0.42, color='C0')
ax[1,0].barh(1.75, ctrl4, 0.42, color='C0')
ax[1,0].barh(1.25, lpd2,  0.42, color='C2')
ax[1,0].barh(0.75, lpd4,  0.42, color='C2')
ax[1,0].set_ylabel('multidecadal', fontsize=30)

ax[1,1].set_xlabel('global OHC variance [ZJ]', fontsize=30)

ctrl1, ctrl2 = std_bandpass(ctrl.OHC_global[-149:])
ctrl3, ctrl4 = std_bandpass(ctrl.OHC_global[:149])
lpd1, lpd2 = std_bandpass(lpd.OHC_global[-149:])
lpd3, lpd4 = std_bandpass(lpd.OHC_global[-2*149:-149])

ax[0,1].barh(2.25, ctrl1/1e21, 0.42, color='C0')
ax[0,1].barh(1.75, ctrl3/1e21, 0.42, color='C0')
ax[0,1].barh(1.25, lpd1 /1e21, 0.42, color='C2')
ax[0,1].barh(0.75, lpd3 /1e21, 0.42, color='C2')

ax[1,1].barh(2.25, ctrl2/1e21, 0.42, color='C0')
ax[1,1].barh(1.75, ctrl4/1e21, 0.42, color='C0')
ax[1,1].barh(1.25, lpd2 /1e21, 0.42, color='C2')
ax[1,1].barh(0.75, lpd4 /1e21, 0.42, color='C2')

print('test')
plt.tight_layout()
plt.savefig(f'{path_results}/poster/std_GMST_OHC', dpi=600)

In [ ]:
def variances(ts):
    for i, band in enumerate([(20,5), (40,10), (80,20)]):
        print(band, bandpass(ts-xr_quadtrend(ts), np.array(band)).std().values)
    print('')
    return
variances(ctrl.OHC_global)
variances(lpd .OHC_global)

variances(gmst_ctrl)
variances(gmst_lpd)
